# Clase Práctica #12 (Compilación)

En esta clase práctica estaremos implementando la fase de análisis semántico para la extensión del lenguaje **xCOOL** presentada en conferencia. Recordemos las reglas del lenguaje.

### Reglas sintácticas

El lenguaje tiene tres tipos de instrucciones: `let`, `def` y `print`:

- `let <var> = <expr>` define una variable denominada `<var>` y le asigna el valor de `<expr>`.
- `def <func>(<arg1>, <arg2>, ...) -> <expr>` define una nueva función `<func>` con los argumentos `<arg*>`.
- `print <expr>` imprime el valor de una expresión.

Las expresiones pueden ser de varios tipos:

- Expresiones aritméticas.
- Invocación de funciones predefinidas (`sin`, `cos`, `pow`, ...).
- Invocación de funciones definidas en el programa.

### Reglas semánticas

- Una variable solo puede ser definida una vez en todo el programa.
- Los nombres de variables y funciones no comparten el mismo ámbito.
- No se pueden redefinir las funciones predefinidas.
- Una función puede tener distintas definiciones siempre que tengan distinta cantidad de argumentos.
- Toda variable y función tiene que haber sido definida antes de ser usada en una expresión (salvo las funciones pre-definidas).
- Todos los argumentos definidos en una misma función tienen que ser diferentes entre sí, aunque pueden ser iguales a variables definidas globalmente o argumentos definidos en otras funciones.
- En el cuerpo de una función, los nombres de los argumentos ocultan los nombres de variables iguales.

## Jerarquía del AST

Comencemos por definir los tipos de nodos que compondrán el AST. Recordemos que estos nodos deben ser capaces de atrapar toda la semántica del programa.

### Nivel 0

En un primer nivel tenemos la clase `Node` que engloba todos los tipos de nodos del AST. Este nodo, a pesar de ser la raíz de la jerarquía, no coincide con la raíz del AST. Los nodos del AST son instancias concretas de las clases definidas en esta jerarquía y siguen una estructura dependiente de la cadena que se parsea.

In [23]:
class Node:
    pass

### Nivel 1

En un segundo nivel tenemos las clases `Program`, `Statement` y `Expression`, siendo estas dos últimas una generalización de varios tipos de instrucciones del lenguaje. `ProgramNode` coincide con la raíz del AST pues representa la base de todo programa en **xCOOL**: una lista de _statements_.

In [24]:
class ProgramNode(Node):
    def __init__(self, statements):
        self.statements = statements
        
class StatementNode(Node):
    pass
        
class ExpressionNode(Node):
    pass

### Nivel 2

Continuamos explorando a lo ancho la jerarquía del AST, y encontramos los 3 tipos de _statements_ en los que se divide un programa: `VarDeclaration` para declarar variables, `FuncDeclaration` para definir métodos, y `PrintNode` para escribir el resultado de evaluar determinada expresión. Además, se proveen dos generalizaciones de expresiones: `AtomicNode` y `BinaryNode`.

In [25]:
class VarDeclarationNode(StatementNode):
    def __init__(self, idx, expr):
        self.id = idx
        self.expr = expr

class FuncDeclarationNode(StatementNode):
    def __init__(self, idx, params, body):
        self.id = idx
        self.params = params
        self.body = body

class PrintNode(StatementNode):
    def __init__(self, expr):
        self.expr = expr

class AtomicNode(ExpressionNode):
    def __init__(self, lex):
        self.lex = lex

class BinaryNode(ExpressionNode):
    def __init__(self, left, right):
        self.left = left
        self.right = right

### Nivel 3

Finalmente alcanzamos el máximo nivel de profundidad, donde tenemos definidos `ConstantNumNode` (representa literales enteros), `VariableNode`  (representa acceso a una variable) y `CallNode` (representa la invocación de un método con determinados argumentos). También encontramos las operaciones aritméticas de suma, resta, multiplicación y división.

In [26]:
class ConstantNumNode(AtomicNode):
    pass

class VariableNode(AtomicNode):
    pass

class CallNode(AtomicNode):
    def __init__(self, idx, args):
        AtomicNode.__init__(self, idx)
        self.args = args

class PlusNode(BinaryNode):
    @staticmethod
    def operate(lvalue, rvalue):
        return lvalue + rvalue

class MinusNode(BinaryNode):
    @staticmethod
    def operate(lvalue, rvalue):
        return lvalue - rvalue

class StarNode(BinaryNode):
    @staticmethod
    def operate(lvalue, rvalue):
        return lvalue

class DivNode(BinaryNode):
    @staticmethod
    def operate(lvalue, rvalue):
        raise NotImplementedError()


## Gramática

Habiendo diseñado la jerarquía de nodos del AST, pasemos a implementar un mecanismo que nos permita construir automáticamente el AST. Nos apoyaremos en las gramáticas atributadas que estudiamos el año pasado para resolver este problema.

Una gramática atributada permite asociar atributos a los símbolos de la gramática y diseñar reglas que los evaluen. Convenientemente, las reglas para construir el AST suelen ser sencillas si la gramática sigue una estructura "natural". Gracias a la implementación de parser LR(1) de la semana anterior, podremos trabajar con una gramática sencilla para este lenguaje, la cual respeta incluso la asociatividad entre los operadores (problema que estuvimos cargando desde el semestre anterior).

Pasemos a construir la gramática e implementar las reglas semánticas.

In [27]:
from cmp.pycompiler import Grammar

G = Grammar()

program = G.NonTerminal('<program>', startSymbol=True)
stat_list, stat = G.NonTerminals('<stat_list> <stat>')
let_var, def_func, print_stat, arg_list = G.NonTerminals('<let-var> <def-func> <print-stat> <arg-list>')
expr, term, factor, atom = G.NonTerminals('<expr> <term> <factor> <atom>')
func_call, expr_list = G.NonTerminals('<func-call> <expr-list>')

let, defx, printx = G.Terminals('let def print')
semi, comma, opar, cpar, arrow = G.Terminals('; , ( ) ->')
equal, plus, minus, star, div = G.Terminals('= + - * /')
idx, num = G.Terminals('id int')

program %= stat_list, lambda h,s: ProgramNode(s[1])

stat_list %= stat + semi, lambda h,s: [s[1]]
stat_list %= stat + semi + stat_list, lambda h,s: [s[1]] + s[3] 

stat %= let_var, lambda h,s: s[1]
stat %= def_func, lambda h,s: s[1]
stat %= print_stat, lambda h,s: s[1]

let_var %= let + idx + equal + expr, lambda h,s: VarDeclarationNode(s[2], s[4])

def_func %= defx + idx + opar + arg_list + cpar + arrow + expr, lambda h,s: FuncDeclarationNode(s[2], s[4], s[7])

print_stat %= printx + expr, lambda h,s: PrintNode(s[2])

arg_list %= idx, lambda h,s: [s[1]]
arg_list %= idx + comma + arg_list, lambda h,s: [s[1]] + s[3]

expr %= expr + plus + term, lambda h,s: PlusNode(s[1], s[3])
expr %= expr + minus + term, lambda h,s: MinusNode(s[1], s[3])
expr %= term, lambda h,s: s[1]

term %= term + star + factor, lambda h,s: StarNode(s[1], s[3])
term %= term + div + factor, lambda h,s: DivNode(s[1], s[3])
term %= factor, lambda h,s: s[1]

factor %= atom, lambda h,s: s[1]
factor %= opar + expr + cpar, lambda h,s: s[2]

atom %= num, lambda h,s: ConstantNumNode(s[1])
atom %= idx, lambda h,s: VariableNode(s[1])
atom %= func_call, lambda h,s: s[1]

func_call %= idx + opar + expr_list + cpar, lambda h,s: CallNode(s[1], s[3])

expr_list %= expr,  lambda h,s: [s[1]]
expr_list %= expr + comma + expr_list, lambda h,s: [s[1]] + s[3] 

print(G)

Non-Terminals:
	<program>, <stat_list>, <stat>, <let-var>, <def-func>, <print-stat>, <arg-list>, <expr>, <term>, <factor>, <atom>, <func-call>, <expr-list>
Terminals:
	let, def, print, ;, ,, (, ), ->, =, +, -, *, /, id, int
Productions:
	[<program> -> <stat_list>, <stat_list> -> <stat> ;, <stat_list> -> <stat> ; <stat_list>, <stat> -> <let-var>, <stat> -> <def-func>, <stat> -> <print-stat>, <let-var> -> let id = <expr>, <def-func> -> def id ( <arg-list> ) -> <expr>, <print-stat> -> print <expr>, <arg-list> -> id, <arg-list> -> id , <arg-list>, <expr> -> <expr> + <term>, <expr> -> <expr> - <term>, <expr> -> <term>, <term> -> <term> * <factor>, <term> -> <term> / <factor>, <term> -> <factor>, <factor> -> <atom>, <factor> -> ( <expr> ), <atom> -> int, <atom> -> id, <atom> -> <func-call>, <func-call> -> id ( <expr-list> ), <expr-list> -> <expr>, <expr-list> -> <expr> , <expr-list>]


## Parseando y evaluando

Saltemos directamente a trabajar con un array de tokens. 
```
print 1 - 1 - 1;
let x = 58;
def f ( a, b ) -> 5 + 6;
print F( 5 + x, 7 + y );
```

El siguiente array resulta de tokenizar al cadena anterior (**ojo:** los espacios en blanco ya fueron eliminados).

In [28]:
from cmp.utils import Token

tokens = [
    Token('print', printx),
    Token('1', num),
    Token('-', minus),
    Token('1', num),
    Token('-', minus),
    Token('1', num),
    Token(';', semi),
    Token('let', let),
    Token('x', idx),
    Token('=', equal),
    Token('58', num),
    Token(';', semi),
    Token('def', defx),
    Token('f', idx),
    Token('(', opar),
    Token('a', idx),
    Token(',', comma),
    Token('b', idx),
    Token(')', cpar),
    Token('->', arrow),
    Token('5', num),
    Token('+', plus),
    Token('6', num),
    Token(';', semi),
    Token('print', printx),
    Token('F', idx),
    Token('(', opar),
    Token('5', num),
    Token('+', plus),
    Token('x', idx),
    Token(',', comma),
    Token('7', num),
    Token('+', plus),
    Token('y', idx),
    Token(')', cpar),
    Token(';', semi),
    Token('$', G.EOF),
]

### Parser LR(1)

Importemos la implementación de parser LR(1) con que trabajamos la clase anterior. Recuerde que si desea ver los estados del autómata LR(1) puede usar la opción `verbose=True` al contruir el parser.

In [29]:
from cmp.tools.parsing import LR1Parser

parser = LR1Parser(G)
parse, operations = parser([t.token_type for t in tokens], get_shift_reduce=True)
parse

[<atom> -> int,
 <factor> -> <atom>,
 <term> -> <factor>,
 <expr> -> <term>,
 <atom> -> int,
 <factor> -> <atom>,
 <term> -> <factor>,
 <expr> -> <expr> - <term>,
 <atom> -> int,
 <factor> -> <atom>,
 <term> -> <factor>,
 <expr> -> <expr> - <term>,
 <print-stat> -> print <expr>,
 <stat> -> <print-stat>,
 <atom> -> int,
 <factor> -> <atom>,
 <term> -> <factor>,
 <expr> -> <term>,
 <let-var> -> let id = <expr>,
 <stat> -> <let-var>,
 <arg-list> -> id,
 <arg-list> -> id , <arg-list>,
 <atom> -> int,
 <factor> -> <atom>,
 <term> -> <factor>,
 <expr> -> <term>,
 <atom> -> int,
 <factor> -> <atom>,
 <term> -> <factor>,
 <expr> -> <expr> + <term>,
 <def-func> -> def id ( <arg-list> ) -> <expr>,
 <stat> -> <def-func>,
 <atom> -> int,
 <factor> -> <atom>,
 <term> -> <factor>,
 <expr> -> <term>,
 <atom> -> id,
 <factor> -> <atom>,
 <term> -> <factor>,
 <expr> -> <expr> + <term>,
 <atom> -> int,
 <factor> -> <atom>,
 <term> -> <factor>,
 <expr> -> <term>,
 <atom> -> id,
 <factor> -> <atom>,
 <ter

Con el objetivo de evaluar los atributos de forma independiente al parseo, se provee la opción `get_shift_reduce=True` al parsear una cadena. Esto indicará al parser, que además del parse derecho (en reverso) nos interesa recuperar la secuencia de operaciones `shift` y `reduce` que aplicó el parser.

In [30]:
print(operations)

['SHIFT', 'SHIFT', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'SHIFT', 'SHIFT', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'SHIFT', 'SHIFT', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'SHIFT', 'SHIFT', 'SHIFT', 'SHIFT', 'SHIFT', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'SHIFT', 'SHIFT', 'SHIFT', 'SHIFT', 'SHIFT', 'SHIFT', 'SHIFT', 'REDUCE', 'REDUCE', 'SHIFT', 'SHIFT', 'SHIFT', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'SHIFT', 'SHIFT', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'SHIFT', 'SHIFT', 'SHIFT', 'SHIFT', 'SHIFT', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'SHIFT', 'SHIFT', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'SHIFT', 'SHIFT', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'SHIFT', 'SHIFT', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'SHIFT', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'SHIFT', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE', 'REDUCE']


### Evaluación

El método `evaluate_reverse_parse` provisto en `cmp.evaluation` nos permitirá evaluar las reglas semánticas de la gramática atributada y obtener la raíz del AST.

In [31]:
from cmp.evaluation import evaluate_reverse_parse

ast = evaluate_reverse_parse(parse, operations, tokens)
ast

## Patrón Visitor

En conferencia discutimos el patrón visitor como una alternativa para implementar recorridos sobre el AST. En lugar de tener funciones declaradas explícitamente en la definición de los nodos (para imprimir el AST, chequear su semántica, evaluarlo, etc.), tendríamos un tipo encargado de hacer cada unos de los recorridos. Esto resulta de gran utilizadad en la confección del compilador, pues permite a partir de fases incrementales ir recopilando información de la semántica del programa, y posteriormente, ir transformándolo a lenguajes cada vez más cercanos al lenguaje destino.

Por ejemplo, a continuación se presenta una implementación de la clase `FormatVisitor`, se que encarga de recorrer el AST y construir una representación como `string` del mismo. Nos apoyamos en el decorador `visitor` provisto en `cmp.visitor`. La decoración `@visitor.on(<parameter_name>)` indica sobre qué parámetro del método `visit` se hará el recorrido. La decoración `@visitor.when(<type>)` indica cuál implementación del método `visit` invocar, según del tipo dinámico del parámetro indicado en `@visitor.on(...)`.

In [32]:
import cmp.visitor as visitor

class FormatVisitor(object):
    @visitor.on('node')
    def visit(self, node, tabs):
        pass
    
    @visitor.when(ProgramNode)
    def visit(self, node, tabs=0):
        ans = '\t' * tabs + f'\\__ProgramNode [<stat>; ... <stat>;]'
        statements = '\n'.join(self.visit(child, tabs + 1) for child in node.statements)
        return f'{ans}\n{statements}'
    
    @visitor.when(PrintNode)
    def visit(self, node, tabs=0):
        ans = '\t' * tabs + f'\\__PrintNode <expr>'
        expr = self.visit(node.expr, tabs + 1)
        return f'{ans}\n{expr}'
    
    @visitor.when(VarDeclarationNode)
    def visit(self, node, tabs=0):
        ans = '\t' * tabs + f'\\__VarDeclarationNode: let {node.id} = <expr>'
        expr = self.visit(node.expr, tabs + 1)
        return f'{ans}\n{expr}'
    
    @visitor.when(FuncDeclarationNode)
    def visit(self, node, tabs=0):
        params = ', '.join(node.params)
        ans = '\t' * tabs + f'\\__FuncDeclarationNode: def {node.id}({params}) -> <expr>'
        body = self.visit(node.body, tabs + 1)
        return f'{ans}\n{body}'

    @visitor.when(BinaryNode)
    def visit(self, node, tabs=0):
        ans = '\t' * tabs + f'\\__<expr> {node.__class__.__name__} <expr>'
        left = self.visit(node.left, tabs + 1)
        right = self.visit(node.right, tabs + 1)
        return f'{ans}\n{left}\n{right}'

    @visitor.when(AtomicNode)
    def visit(self, node, tabs=0):
        return '\t' * tabs + f'\\__ {node.__class__.__name__}: {node.lex}'
    
    @visitor.when(CallNode)
    def visit(self, node, tabs=0):
        ans = '\t' * tabs + f'\\__CallNode: {node.lex}(<expr>, ..., <expr>)'
        args = '\n'.join(self.visit(arg, tabs + 1) for arg in node.args)
        return f'{ans}\n{args}'

Construir una instancia de `FormatVisitor` y visitar la raíz del AST tiene el efecto siguiente.

In [33]:
formatter = FormatVisitor()
print(formatter.visit(ast))

\__ProgramNode [<stat>; ... <stat>;]
	\__PrintNode <expr>
		\__<expr> MinusNode <expr>
			\__<expr> MinusNode <expr>
				\__ ConstantNumNode: 1
				\__ ConstantNumNode: 1
			\__ ConstantNumNode: 1
	\__VarDeclarationNode: let x = <expr>
		\__ ConstantNumNode: 58
	\__FuncDeclarationNode: def f(a, b) -> <expr>
		\__<expr> PlusNode <expr>
			\__ ConstantNumNode: 5
			\__ ConstantNumNode: 6
	\__PrintNode <expr>
		\__CallNode: F(<expr>, ..., <expr>)
			\__<expr> PlusNode <expr>
				\__ ConstantNumNode: 5
				\__ VariableNode: x
			\__<expr> PlusNode <expr>
				\__ ConstantNumNode: 7
				\__ VariableNode: y


## Contexto

Para chequear la semántica del lenguaje en cuestión, resulta necesario registrar qué variables han sido declaradas y qué métodos han sido definidos. Utilizaremos las siguientes clases como contenedores de la información "relevante" por ahora de las variables y métodos definidos. De las variables nos interesa su nombre y de las funciones su nombre y el de sus parámetros.

In [34]:
class VariableInfo:
    def __init__(self, name):
        self.name = name

class FunctionInfo:
    def __init__(self, name, params):
        self.name = name
        self.params = params

Dado que la visibilidad de las variables depende del contexto, será necesario incluir un mecanismo para ocultar (1) variables definidas en ámbitos más profundos al actual, o (2) en ámbitos más hacia adelante que el actual. Para resolver lo primero, haremos que nuestra clase contexto, o ámbito, tenga una estructura jerárquica: cada nodo tiene un padre y potencialmente múltiples hijos. Para resolver lo segundo, será necesario marcar un índice al crear scopes hijos, el cual indique hasta qué sección del ambito padre deberían poder buscar (por ejemplo, no se deberían poder ver variables definidas en el padre más adelantes del momento en que se creó el scope hijo).

Teniendo en cuenta estas restricciones, la búsqueda de un registro en el scope debería ocurrir como una búsqueda local en el scope en cuestión, pero continuada hacia el scope padre (y así consecutivamente) si no se encontró definición local.

In [35]:
import itertools as itl

class Scope:
    def __init__(self, parent=None):
        self.local_vars = []
        self.local_funcs = []
        self.parent = parent
        self.children = []
        self.var_index_at_parent = 0 if parent is None else len(parent.local_vars)
        self.func_index_at_parent = 0 if parent is None else len(parent.local_funcs)
        
    def create_child_scope(self):
        child_scope = Scope(self)
        self.children.append(child_scope)
        return child_scope

    def define_variable(self, vname):
        if self.is_var_defined(vname):
            return False
        else:
            self.local_vars.append(VariableInfo(vname))
            return True
    
    def define_function(self, fname, params):
        if self.is_func_defined(fname, len(params)):
            return False
        else:
            self.local_funcs.append(FunctionInfo(fname, params))
            return True

    def is_var_defined(self, vname):
        return self.is_local_var(vname) or (self.parent is not None and self.parent.is_var_defined(vname))
    
    def is_func_defined(self, fname, n):
        return self.is_local_func(fname, n) or (self.parent is not None and self.parent.is_func_defined(fname, n))

    def is_local_var(self, vname):
        return self.get_local_variable_info(vname) is not None
    
    def is_local_func(self, fname, n):
        return self.get_local_function_info(fname, n) is not None

    def get_local_variable_info(self, vname):
        try:
            return next(filter(lambda x: x.name == vname, self.local_vars))
        except StopIteration:
            return
    
    def get_local_function_info(self, fname, n):
        try:
            return next(filter(lambda x: x.name == fname and len(x.params) == n, self.local_funcs))
        except StopIteration:
            return
    
    
scope = Scope()

## Chequeo semántico

Finalmente, implementemos un visitor para chequear las reglas semánticas del lenguaje discutidas a inicios de la clase práctica.

In [36]:
class SemanticCheckerVisitor(object):
    def __init__(self):
        self.errors = []
    
    @visitor.on('node')
    def visit(self, node, scope):
        pass
    
    @visitor.when(ProgramNode)
    def visit(self, node, scope=None):
        scope = Scope()
        for statement in node.statements:
            self.visit(statement, scope)
        return self.errors
    
    @visitor.when(VarDeclarationNode)
    def visit(self, node, scope):
        if scope.define_variable(node.id):
            self.visit(node.expr, scope)
        else:
            self.errors.append(f'Varible {node.id} declared before')
            
    @visitor.when(FuncDeclarationNode)
    def visit(self, node, scope):
        if scope.define_function(node.id, node.params):
            child_scope = scope.create_child_scope()

            for param in node.params:
                child_scope.define_variable(param)

            self.visit(node.body, child_scope)
        else:
            self.errors.append(f'Varible {node.id} declared before')
    
    @visitor.when(PrintNode)
    def visit(self, node, scope):
        self.visit(node.expr, scope)
    
    @visitor.when(ConstantNumNode)
    def visit(self, node, scope):
        return
    
    @visitor.when(VariableNode)
    def visit(self, node, scope):
        if not scope.is_var_defined(node.lex):
            self.errors.append(f'Invalid variable: {node.lex}')
    
    @visitor.when(CallNode)
    def visit(self, node, scope):
        if not scope.is_func_defined(node.lex, len(node.args)):
            self.errors.append(f'Function {node.lex} is not defined with {len(node.args)} arguments.')
        
        for arg in node.args:
            self.visit(arg, scope)
        
    @visitor.when(BinaryNode)
    def visit(self, node, scope):
        self.visit(node.left, scope)
        self.visit(node.right, scope)


### Chequeo

Deberían detectarse como mínimo 2 errores:

0. Function F is not defined with 2 arguments.
1. Invalid variable: y.

In [37]:
semantic_checker = SemanticCheckerVisitor()
errors = semantic_checker.visit(ast)
for i, error in enumerate(errors, 1):
    print(f'{i}.', error)

1. Function F is not defined with 2 arguments.
2. Invalid variable: y


## Propuestas

- Incluya línea y columna en los errores detectados.
- Incluya el lexer.
- Construya una clase que siga el patrón visitor para evaluar / ejecutar el programa.

### Incluyendo el lexer

In [38]:
from lexer import Lexer

In [39]:
digits1_9 = '|'.join([str(i) for i in range(1, 10)])
digits0_9 = '0|' + digits1_9
chars = '|'.join([chr(i) for i in range(97, 123)] + [chr(i) for i in range(65, 91)])

lexer = Lexer([
    (num, f'({digits1_9})({digits0_9})*'),
    (plus, '\+'),
    (minus, '-'),
    (star, '\*'),
    (div, '/'),
    (semi, ';'),
    (comma, ','),
    (opar, '\('),
    (cpar, '\)'),
    (equal, '='),
    (arrow, '->'),
    (printx, 'print'),
    (let, 'let'),
    (defx, 'def'),
    ('space', '  *'),
    ('newline', '\n\n*'),
    (idx, f'({chars})({chars}|{digits0_9})*'),
], G.EOF)

In [40]:
program = '''
print 1 - 1; 
let x = 58; 
def sum(a, b) -> a + b; 
print sum(5 + x, 7 + x);
'''

tokens = lexer(program)
tokens = [t for t in tokens if t.token_type not in ('space', 'newline')]

In [41]:
parser = LR1Parser(G)
parse, operations = parser([t.token_type for t in tokens], get_shift_reduce=True)
ast = evaluate_reverse_parse(parse, operations, tokens)
print(formatter.visit(ast))

\__ProgramNode [<stat>; ... <stat>;]
	\__PrintNode <expr>
		\__<expr> MinusNode <expr>
			\__ ConstantNumNode: 1
			\__ ConstantNumNode: 1
	\__VarDeclarationNode: let x = <expr>
		\__ ConstantNumNode: 58
	\__FuncDeclarationNode: def sum(a, b) -> <expr>
		\__<expr> PlusNode <expr>
			\__ VariableNode: a
			\__ VariableNode: b
	\__PrintNode <expr>
		\__CallNode: sum(<expr>, ..., <expr>)
			\__<expr> PlusNode <expr>
				\__ ConstantNumNode: 5
				\__ VariableNode: x
			\__<expr> PlusNode <expr>
				\__ ConstantNumNode: 7
				\__ VariableNode: x


In [42]:
semantic_checker = SemanticCheckerVisitor()
errors = semantic_checker.visit(ast)
not errors

True

### Clase que sigue el patrón visitor para evaluar / ejecutar el programa.

In [43]:
class ExecVisitor(object):
    def __init__(self):
        self.errors = []
    
    @visitor.on('node')
    def visit(self, node, scope):
        pass
    
    @visitor.when(ProgramNode)
    def visit(self, node, scope=None):
        scope = Scope()
        for stat in node.statements:
            self.visit(stat, scope)

    @visitor.when(VarDeclarationNode)
    def visit(self, node, scope):
        value = self.visit(node.expr, scope)
        scope.define_variable(node.id)
        var_info = scope.get_local_variable_info(node.id)
        var_info.value = value
            
    @visitor.when(FuncDeclarationNode)
    def visit(self, node, scope):
        scope.define_function(node.id, node.params)
        func_info = scope.get_local_function_info(node.id, len(node.params))
        func_info.body = node.body 
    
    @visitor.when(PrintNode)
    def visit(self, node, scope):
        print(self.visit(node.expr, scope))
    
    @visitor.when(ConstantNumNode)
    def visit(self, node, scope):
        return float(node.lex)
    
    @visitor.when(VariableNode)
    def visit(self, node, scope):
        current_scope = scope
        
        info = None
        while info is None:
            info = current_scope.get_local_variable_info(node.lex)
            current_scope = current_scope.parent
        
        return info.value
    
    @visitor.when(CallNode)
    def visit(self, node, scope):
        current_scope = scope
        
        info = None
        while info is None:
            info = current_scope.get_local_function_info(node.lex, len(node.args))
            current_scope = current_scope.parent
        
        child_scope = scope.create_child_scope()
        
        for name, param in zip(info.params, node.args):
            value = self.visit(param, scope)
            child_scope.define_variable(name)
            child_scope.get_local_variable_info(name).value = value
        
        return self.visit(info.body, child_scope)
        

    @visitor.when(PlusNode)
    def visit(self, node, scope):
        return self.visit(node.left, scope) + self.visit(node.right, scope)
    
    @visitor.when(MinusNode)
    def visit(self, node, scope):
        return self.visit(node.left, scope) - self.visit(node.right, scope)
    
    @visitor.when(StarNode)
    def visit(self, node, scope):
        return self.visit(node.left, scope) * self.visit(node.right, scope)
    
    @visitor.when(DivNode)
    def visit(self, node, scope):
        return self.visit(node.left, scope) / self.visit(node.right, scope)


### Corriendo el programa

In [44]:
executor = ExecVisitor()
executor.visit(ast)

0.0
128.0
